<a href="https://colab.research.google.com/github/shjiinn/Python_Review/blob/master/%EC%95%84%ED%8C%8C%EC%B9%98_%ED%95%98%EB%91%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 아파치 하둡

1. 하둡
  - 비정형 데이터를 포함한 빅데이터를 다루기 위한 가장 적절한 플랫폼
  - open 소스
  - 하둡 에코시스템
  - 저장하기 위해 큰 비용이 들지 않음

2. HADOOP 탄생
  - 인덱싱 라이브러리 Lucene 오픈소스로 공개
  - Nutch(웹 검색엔진 프로젝트) 탄생 시킴
  - Hadoop(빅데이터 처리 프로젝트) 탄생

3. 하둡 분산파일시스템(HDFS) 이해
- 하둡 특성
  - 수천대 이상의 리눅스 기반 범용 서버들을 하나의 클러스터로 사용
  - master-slave 구조 : master에 안정성을 신경써야함
  - 파일은 블록 단위로 저장
  - 블록 데이터의 복제본 유지로 인한 신뢰성 보장(기본 3개의 복제본)
  - 높은 내고장성(Fault-Tolerance)
  - 데이터 처리의 지역성 보장

- 하둡의 블록
  - 하나의 파일을 여러 개의 block으로 저장
  - 설정에 의해 하나의 block은 64mb or 128mb 등의 큰 크기로 나누어 저장
  - 블록 크기가 128mb보다 적은 경우는 실제 크기 만큼만 용량을 차지함
  - 블록 하나의 크기가 큰 이유
    - HDFS의 블록은 128mb와 같이 매우 큰 단위
    - 블록이 큰 이유 : 탐색 비용을 최소화하기 위해
    - 블록이 크면 하드디스크에서 블록의 시작점을 탐색하는 데 걸리는 시간을 줄일 수 있고, 네트워크를 통해서 데이터를 전송하는데 더 많은 시간을 할당 가능
  - 저장 특징
    - 데이터를 조각내어 서버내 분산 저장
    - 데이터를 복사하여 여러개를 저장
    - Name node(Master)
      - 데이터의 위치, 형식 보관
    - Data node(Slave)
      - 실 데이터 저장
    - 플래폼 자체적으로 3개씩 저장함 
  - 블록의 지역성(Locality)
    - 네트워크를 이용한 데이터 전송 시간 감소
    - 대용량 데이터 확인을 위한 디스크 탐색 시간 감소
    - 적절한 단위의 블록 크기를 이용한 CPU 처리시간 증가
  - 블록 캐싱
    - 데이터 노드에 저장된 데이터 중 자주 읽는 블록은 블록 캐시라는 데이터 노드의 메모리에 명시적으로 캐싱할 수 있음
    - 파일 단위로 캐싱할 수도 있어서 조인에 사용되는 데이터들을 등록하여 읽기 성능을 높일 수 있음

  - 네임노드 역할
    - 전체 HDFS에 대한 Name Space 관리
    - DataNode로 부터 Block 리포트를 받음
    - Data에 대한 Replication 유지를 위한 커맨더 역할 수행
    - 파일 시스템 이미지 파일관리(fsimage)
      - fsimage가 손상되면 하둡이 다 날라갈수도?
      - 스냅샷
    - 파일 시스템에 대한 Edit Log 관리
  - 보조 네임노드(SNN)
    - 네임노드와 보조 네임노드
      - Active/Standby 구조 아님
      - fsimage와 edits 파일을 주기적으로 병합
    - 체크 포인트
      - 1시간 주기로 실행
      - edits 로그가 일정 사이즈 이상이면 실행
    - 이슈사항
      - 네임노드가 SPOF
      - 보조 네임노드의 장애 상황 감지 툴 없음
    - 네임 노드는 늘 최신상태 유지 -> 데몬을 구동 -> fsimage를 읽음 -> 메모리에 스냅샷(전체 하드 클러스터에 있는 파일 베타 정보들을 구성) -> edit Log를 읽어가며 변경된 내용을 메모리에 반영 -> 서비스 시작 -> 시스템의 변경이 일어났을 경우 전부 edit Log에 기록이 남음(edit Log의 용량이 늘어남) -> edit Log를 fsimage와 병합을 계속 해줘야함 -> 병합 시 보조네임노드에 두 파일을 보내고 merge(병합)을 한 후에 fsimage를 바꿔 부는 작업을 주기적으로 돌아가면서 최근의 메모리 상태 정보와 거의 유사한 형태를 유지, editLog의 양이 최소한으로 줄어들 수 있도록 계속 merge 해줌
    - 보조 네임노드(Second Name Node, SNN) 장애 발생
      - 아무 문제가 발생하지 않음
      - 운영관점에서 editsLog가 무한히 커지는 문제 발생 -> 리스타트 시 파일이 너무 크면 읽지 못하고 Out of Memory Reception이 발생할 수 있음
    - 데이터노드 역할
      - DataNode는 물리적으로 로컨 파일시스템에 HDFS 데이터를 저장
      - DataNode는 HDFS에 대한 지식이 없음
      - 일반적으로 레이드 구성을 하지 않음(JBOD 구성)
      - 블록 리포트 : NameNode가 시작될 때, 그리고 주기적으로 로컬 파일시스템에 있는 모든 HDFS 블록들을 검사 후 정상적인 블록의 목록을 만들어 NameNode에 전송